## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
import os
print(os.getcwd())

C:\Users\lmsul\Class\World_Weather_Analysis\Vacation_Search


In [3]:
print(os.listdir(os.getcwd())

SyntaxError: unexpected EOF while parsing (<ipython-input-3-fca77e0221d4>, line 1)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fort Saint James,CA,54.4333,-124.2500,29.10,95,100,2.46,overcast clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,-4.00,77,90,11.50,light snow
2,2,New Norfolk,AU,-42.7826,147.0587,50.00,66,25,3.00,scattered clouds
3,3,Atuona,PF,-9.8000,-139.0333,77.45,77,1,13.27,clear sky
4,4,Kavaratti,IN,10.5669,72.6420,82.27,77,10,9.42,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 6
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fort Saint James,CA,54.4333,-124.2500,29.10,95,100,2.46,overcast clouds
2,2,New Norfolk,AU,-42.7826,147.0587,50.00,66,25,3.00,scattered clouds
3,3,Atuona,PF,-9.8000,-139.0333,77.45,77,1,13.27,clear sky
4,4,Kavaratti,IN,10.5669,72.6420,82.27,77,10,9.42,clear sky
5,5,East London,ZA,-33.0153,27.9116,69.80,68,0,27.63,clear sky
6,6,Mataura,NZ,-46.1927,168.8643,63.00,85,100,3.00,overcast clouds
7,7,Punta Arenas,CL,-53.1500,-70.9167,60.80,59,90,14.97,overcast clouds
10,10,Ushuaia,AR,-54.8000,-68.3000,59.00,63,75,4.61,broken clouds
11,11,Mahebourg,MU,-20.4081,57.7000,82.40,78,40,8.05,scattered clouds
12,12,Hilo,US,19.7297,-155.0900,69.80,60,90,8.05,overcast clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Fort Saint James,CA,29.10,overcast clouds,54.4333,-124.2500,
2,New Norfolk,AU,50.00,scattered clouds,-42.7826,147.0587,
3,Atuona,PF,77.45,clear sky,-9.8000,-139.0333,
4,Kavaratti,IN,82.27,clear sky,10.5669,72.6420,
5,East London,ZA,69.80,clear sky,-33.0153,27.9116,
6,Mataura,NZ,63.00,overcast clouds,-46.1927,168.8643,
7,Punta Arenas,CL,60.80,overcast clouds,-53.1500,-70.9167,
10,Ushuaia,AR,59.00,broken clouds,-54.8000,-68.3000,
11,Mahebourg,MU,82.40,scattered clouds,-20.4081,57.7000,
12,Hilo,US,69.80,overcast clouds,19.7297,-155.0900,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print ("Hotel not found ... skipping")
        

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not foun

In [35]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [30]:
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Fort Saint James,CA,29.10,overcast clouds,54.4333,-124.2500,The View Hotel
2,New Norfolk,AU,50.00,scattered clouds,-42.7826,147.0587,The Shingles Riverside Cottages
3,Atuona,PF,77.45,clear sky,-9.8000,-139.0333,Villa Enata
4,Kavaratti,IN,82.27,clear sky,10.5669,72.6420,Hotels in Lakshadweep Islands
5,East London,ZA,69.80,clear sky,-33.0153,27.9116,Tu Casa
6,Mataura,NZ,63.00,overcast clouds,-46.1927,168.8643,Ellie's Villa
7,Punta Arenas,CL,60.80,overcast clouds,-53.1500,-70.9167,Hotel Dreams Del Estrecho
10,Ushuaia,AR,59.00,broken clouds,-54.8000,-68.3000,Albatros Hotel
11,Mahebourg,MU,82.40,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
12,Hilo,US,69.80,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [ ]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
